RAIN - Real & Artificial Intelligence in Neuroscience

## Draw ROIs

How to use:
- Select the videos you want to draw on
- Left-click to select a point
- Left-click and drag to draw a rectangle
  - Right-click and drag to move the rectangle
  - Use the scroll wheel to resize the rectangle
  - Use Ctrl + scroll wheel to rotate the rectangle
- Press 'S' to save the current ROI
- Press 'Q' to quit and save all ROIs

In [1]:
import os
import cv2
import json
from tkinter import Tk, filedialog, messagebox, simpledialog
import numpy as np  # Needed for rotation matrix computations

def define_rectangle(x1, y1, x2, y2):
                width = int(round(abs(x2 - x1)))
                height = int(round(abs(y2 - y1)))
                center = [int(round((x1+x2)//2)), int(round((y1+y2)//2))] # Round to integer
                return center, width, height

def draw_rectangle(image, center, width, height, rotation = 0, color = (0, 255, 0), thickness = 2):
    angle = rotation
    box = cv2.boxPoints(((center[0], center[1]), (width, height), angle))
    box = np.int0(box)  # Convert to integer
    cv2.drawContours(image, [box], 0, color, thickness)
    cv2.circle(image, (int(round(center[0])), int(round(center[1]))), radius=2, color=color, thickness=-1)

def merge_frames(video_files: list) -> np.ndarray:
    """
    Merge the first frames of user-selected videos with transparency.
    
    Returns:
        np.ndarray: Merged image.
    """
    merged_image = None
    
    for video_file in video_files:
        cap = cv2.VideoCapture(video_file)
        success, frame = cap.read()
        cap.release()
        
        if not success:
            print(f"Could not read first frame of {video_file}")
            continue
        
        # Calculate transparency
        transparency = round(1 / len(video_files), 4)
        transparent_frame = (frame * transparency).astype(np.uint8)
        
        if merged_image is None:
            # Initialize merged image
            merged_image = np.zeros_like(transparent_frame)
        
        # Add transparent frame to the merged image
        merged_image = cv2.add(merged_image, transparent_frame)
    
    return merged_image

# Main function to handle ROI selection
def roi_selector():

    print("Instructions:")
    print("1. Left-click and drag to draw a rectangle.")
    print("2. Right-click and drag to move the rectangle.")
    print("3. Use the scroll wheel to resize the rectangle.")
    print("4. Use Ctrl + scroll wheel to rotate the rectangle.")
    print("5. Press 'S' to save the current ROI.")
    print("6. Press 'Q' to quit and save all rectangles to the CSV file.")

    # Initialize Tkinter and hide the root window
    root = Tk()
    root.withdraw()
    
    # Open file dialog to select video files
    video_files = filedialog.askopenfilenames(
        title="Select Video Files",
        filetypes=[("Video Files", "*.mp4 *.avi *.mkv *.mov")]
    )
    if not video_files:
        raise ValueError("No video files selected.")
    
    print(f"Selected {len(video_files)} videos.")

    # Merge frames from the selected videos
    image = merge_frames(video_files)
    
    # Create metadata dictionary
    video_metadata = {
        "frame_shape": {"width": image.shape[1], "height": image.shape[0]},
        "rois": []  # List of ROIs
    }

    # Initialize variables
    clone = image.copy()
    corners = []  # Current ROI 
    dragging = [False]  # For moving ROI
    drag_start = None
    current_angle = 0  # Current angle for rotation
    rotate_factor = 1  # Amount of change per scroll
    resize_factor = 2  # Amount of change per scroll

    # Mouse callback function
    def handle_mouse(event, x, y, flags, param):
        nonlocal video_metadata, clone, corners, dragging, drag_start, current_angle, rotate_factor, resize_factor

        # Start drawing the rectangle
        if event == cv2.EVENT_LBUTTONDOWN:
            # current_angle = 0 # Reset angle
            if not dragging[0]:
                dragging[0] = True
                corners = [(x, y)]  # Start new ROI at the clicked position

        # Update rectangle during drawing
        elif event == cv2.EVENT_MOUSEMOVE and dragging[0] and len(corners) == 1:
            clone = image.copy()

            for r in video_metadata["rois"]:
                draw_rectangle(clone, r["center"], r["width"], r["height"], r["angle"])
            
            x1, y1 = corners[0]
            x2, y2 = x, y # While dragging, update the end point
            center, width, height = define_rectangle(x1, y1, x2, y2)
            draw_rectangle(clone, center, width, height, current_angle, (0, 255, 255), 2)
            
        # Finish drawing the rectangle
        elif event == cv2.EVENT_LBUTTONUP:
            if dragging[0]:
                dragging[0] = False
                corners.append((x, y))  # End point

        # Start moving the rectangle
        elif event == cv2.EVENT_RBUTTONDOWN and len(corners) == 2:
            dragging[0] = True
            drag_start = (x, y)

        # Move the rectangle
        elif event == cv2.EVENT_MOUSEMOVE and dragging[0] and len(corners) == 2:
            dx = x - drag_start[0]
            dy = y - drag_start[1]
            drag_start = (x, y)
            corners[0] = (corners[0][0] + dx, corners[0][1] + dy)
            corners[1] = (corners[1][0] + dx, corners[1][1] + dy)
            clone = image.copy()

            for r in video_metadata["rois"]:
                draw_rectangle(clone, r["center"], r["width"], r["height"], r["angle"])
            
            x1, y1 = corners[0]
            x2, y2 = corners[1]
            center, width, height = define_rectangle(x1, y1, x2, y2)
            draw_rectangle(clone, center, width, height, current_angle, (0, 255, 255), 2)

        # Stop moving the rectangle
        elif event == cv2.EVENT_RBUTTONUP and len(corners) == 2:
            dragging[0] = False

        # Resize or rotate the ROI using scroll wheel
        elif event == cv2.EVENT_MOUSEWHEEL and len(corners) == 2:
            if flags & cv2.EVENT_FLAG_CTRLKEY:  # Rotate with Ctrl key pressed
                if flags > 0:  # Scroll up
                    current_angle -= rotate_factor
                else:  # Scroll down
                    current_angle += rotate_factor
            else:  # Resize without modifier key
                x1, y1 = corners[0]
                x2, y2 = corners[1]
                width = max(abs(x2 - x1), 1)
                height = max(abs(y2 - y1), 1)
                ratio = width/height
                if flags > 0:  # Scroll up
                    x1 -= resize_factor*ratio
                    y1 -= resize_factor
                    x2 += resize_factor*ratio
                    y2 += resize_factor
                else:  # Scroll down
                    x1 += resize_factor*ratio
                    y1 += resize_factor
                    x2 -= resize_factor*ratio
                    y2 -= resize_factor
                corners = [(x1, y1), (x2, y2)]
                
            clone = image.copy()
            for r in video_metadata["rois"]:
                draw_rectangle(clone, r["center"], r["width"], r["height"], r["angle"])
            
            x1, y1 = corners[0]
            x2, y2 = corners[1]
            center, width, height = define_rectangle(x1, y1, x2, y2)
            draw_rectangle(clone, center, width, height, current_angle, (0, 255, 255), 2)

        # Draw the updated ROI and display width, height, and angle
        if len(corners) > 0:
            x1, y1 = corners[0]
            if len(corners) > 1:
                x2, y2 = corners[1]
            else:
                x2, y2 = x, y     
            
            center, width, height = define_rectangle(x1, y1, x2, y2)

            # Display height, width, and angle at the bottom of the frame
            text = f"Center: {center}, W: {width}, H: {height}, A: {current_angle}"  # Convert to int for display
            font_scale = 0.5
            font_thickness = 1
            text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)[0]
            text_x = 10
            text_y = clone.shape[0] - 10
            cv2.rectangle(clone, (text_x - 5, text_y - text_size[1] - 5), 
                        (text_x + text_size[0] + 5, text_y + 5), (0, 0, 0), -1)  # Background for text
            cv2.putText(clone, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 
                        font_scale, (255, 255, 255), font_thickness)

    # Set up the window and mouse callback
    cv2.namedWindow("Select ROIs")
    cv2.setMouseCallback("Select ROIs", handle_mouse)

    while True:
        cv2.imshow("Select ROIs", clone)
        key = cv2.waitKey(1) & 0xFF
            
        if key == ord('s') and len(corners) == 2:  # Save the ROI
            name = simpledialog.askstring("Input", "Enter a name for the ROI:")
            if not name:
                print(f"ROI not saved.")
            else:
                x1, y1 = corners[0]
                x2, y2 = corners[1]
                center, width, height = define_rectangle(x1, y1, x2, y2)
                saved_roi = {
                    "name": name,
                    "center": center,
                    "width": width,
                    "height": height,
                    "angle": current_angle
                }
                video_metadata["rois"].append(saved_roi)
                print(f"Saved ROI: {saved_roi}")

                clone = image.copy()
                for r in video_metadata["rois"]:
                    draw_rectangle(clone, r["center"], r["width"], r["height"], r["angle"])

        elif key == ord('q'):  # Quit and save
            response = messagebox.askquestion("Exit", "Do you want to exit the ROI selector?")
            if response == 'yes':
                response = messagebox.askquestion("Exit", "Do you want to save ROIs?")
                if response == 'yes':
                    save = True
                else:
                    save = False
                break

    cv2.destroyAllWindows()

    # Save the ROIs to a CSV file
    if save:
        output_json = os.path.join(os.path.dirname(video_files[0]), 'ROIs.json')
        with open(output_json, 'w') as file:
            json.dump(video_metadata, file, indent=4)

        print(f"ROIs saved to {output_json}")
    
    else:
        print("No ROIs saved.")


In [2]:
# Example usage:
roi_selector()

Instructions:
1. Left-click and drag to draw a rectangle.
2. Right-click and drag to move the rectangle.
3. Use the scroll wheel to resize the rectangle.
4. Use Ctrl + scroll wheel to rotate the rectangle.
5. Press 'S' to save the current ROI.
6. Press 'Q' to quit and save all rectangles to the CSV file.
Selected 18 videos.


C:\Users\dhers\AppData\Local\Temp\ipykernel_12976\47941107.py:16: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)  # Convert to integer


Saved ROI: {'name': 'closed_arms', 'center': [445, 445], 'width': 888, 'height': 90, 'angle': 0}
Saved ROI: {'name': 'open_arms', 'center': [445, 445], 'width': 770, 'height': 78, 'angle': 90}
ROIs saved to C:/Users/dhers/OneDrive/Doctorado/Experimentos/3xTg_B2/2024_04-E+M/resized/Aligned/cropped\ROIs.json
